The `Estimator` APIs are a high-level API in Tensorflow or say a high-level representation of a model. It is designed for easy scaling and asynchronous training. 

In [0]:
!pip install tf-nightly

In [4]:
import tensorflow as tf
import pandas as pd

print("Tensorflow Version: {}".format(tf.__version__))
print("Eager Model: {}".format(tf.executing_eagerly()))
print("GPU {} available".format("is" if tf.config.experimental.list_physical_devices("GPU") else "not"))

Tensorflow Version: 2.2.0-dev20200112
Eager Model: True
GPU is available


# The Dataset

In this tutorial, you are going to classify Iris flowers and use a famous Iris dataset. You will use 4 features (sepal length, sepal width, petal length, petal width) to predict 1 class (Setosa, Versicolor, Virginica).

In [0]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [6]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", 
    "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", 
    "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


You can inspect the dataset first.

In [7]:
train.head(2)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1


The column `Species` is the target for training.

In [0]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [9]:
train.head(2)

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0


# Modeling via Estimators

An Estimator is from the class `tf.estimator.Estimator`. Tensorflow provides a collection of `tf.estimator`, e.g. `LinearRegressor`, to implement common ML algorithms. You can also customize your own estimators. The basic steps are:

* Create one or more inputs.
* Define the model's feature columns.
* Instantiate an Estimator, specifying the feature columns and various hyperparameters.
* Call one or more methods on the Estimator object, including a loss function, a basic estimator, etc.
* Pass the appropriate input function and feature columns.

## Creating Input Functions

An input function is used for feeding the dataset into the model while training, evaluating and predicting. In general, an input function must return two elements, `features` and `labels`, in a tuple. The simple input function is composed of:
* `features`: a Python dictionary object
* `labels`: the value of the label for every example

The following is a simple implementation.

In [0]:
def input_evaluation_set():
  features = {
    'SepalLength': np.array([6.4, 5.0]),
    'SepalWidth': np.array([2.8, 3.3]),
    'PetalLength': np.array([5.6, 3.3]),
    'PetalWidth': np.array([2.2, 1.8])
  }
  labels = np.array([2, 1])
  return features, labels

The recommended way of implementing an input function is the one by `tf.data.Dataset`. These APIs are not only supporting multiple types of data sources but also implemented on parallel pipelines and efficiently loading large datasets.

In [0]:
def input_fn(features, labels, training=True, batch_size=256):
  """An input function for training and evaluating."""
  # convert the inputs to a Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # shuffle and repeat if you are in training mode
  if training:
    dataset = dataset.repeat().shuffle(1000)
  
  return dataset.batch(batch_size)

## Defining the Feature Columns

A feature column is an object describing how the model should use raw input data from the dictionary features dictionary.

In [15]:
feature_columns = []
for key in train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key=key))
feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Instantiating an Estimator

Now you are going to create an estimator, in Tensorflow there are lots of built-in estimators.

* `tf.estimator.DNNClassifier`: performs multi-class classification
* `tf.estimator.DNNLinearCombinedClassifier`
* `tf.estimator.LinearClassifier`: classification based on linear models

... (more)

For this problem, you can instantiate a built-in estimator `DNNClassifier` to predict a multi-class classification.

In [17]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns, 
    # build a two-layer DNN model, each layer is composed of 30 and 10 nodes respectively
    hidden_units=[30, 10],
    n_classes=len(SPECIES))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpidz426vh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train, Evaluate and Predict

### Train the Model

You can call the method `.train()` to start training. Here you are going to wrap the input_fn with a `lambda` function for feeding the dataset and the label due to no dataset provided on the input_fn.

In [19]:
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True),
                 steps=5000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

### Evaluate the Model

In [20]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))
print("Test dataset accuracy: {accuracy:0.3f}".format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-13T10:25:28Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpidz426vh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27777s
INFO:tensorflow:Finished evaluation at 2020-01-13-10:25:28
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.53333336, averag

### Make a Prediction

In [0]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
  'SepalLength': [5.1, 5.9, 6.9],
  'SepalWidth': [3.3, 3.0, 3.1],
  'PetalLength': [1.7, 4.2, 5.4],
  'PetalWidth': [0.5, 1.5, 2.1],
}

def pred_input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [0]:
predictions = classifier.predict(input_fn=lambda: pred_input_fn(predict_x))

In [27]:
for pred_dict, expect in zip(predictions, expected):
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print("Prediction: {}, Prob: {}, Expected: {}".format(
      SPECIES[class_id], probability, expect))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpidz426vh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction: Setosa, Prob: 0.661624550819397, Expected: Setosa
Prediction: Virginica, Prob: 0.4341447055339813, Expected: Versicolor
Prediction: Virginica, Prob: 0.5762437582015991, Expected: Virginica
